In [59]:
import import_ipynb
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utilities.UtilityFunctions import retrieve_reviews_df, preprocessing, vectorize, text_preprocess, retrieve_processed_reviews_df

from statsmodels.miscmodels.ordinal_model import OrderedModel

In [60]:
df = retrieve_processed_reviews_df()

Concatenating ../processed_data\processed_data-1.csv
Concatenating ../processed_data\processed_data-2.csv
Concatenating ../processed_data\processed_data-3.csv
Concatenating ../processed_data\processed_data-4.csv
Concatenating ../processed_data\processed_data-5.csv


In [33]:
df

,Id,Score,Summary,Text,Sentiment,Clean_text
0,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,i have bought several of the vitality canned d...
1,2,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,-1,product arrived labeled as jumbo salted peanut...
2,3,4,"""Delight"" says it all",This is a confection that has been around a fe...,0,this is a confection that has been around a fe...
3,4,2,Cough Medicine,If you are looking for the secret ingredient i...,-1,if you are looking for the secret ingredient i...
4,5,5,Great taffy,Great taffy at a great price. There was a wid...,1,great taffy at a great price there was a wide...
...,...,...,...,...,...,...
568449,568450,5,Will not do without,Great for sesame chicken..this is a good if no...,1,great for sesame chickenthis is a good if not ...
568450,568451,2,disappointed,I'm disappointed with the flavor. The chocolat...,-1,im disappointed with the flavor the chocolate ...
568451,568452,5,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o...",1,these stars are small so you can give 1015 of ...
568452,568453,5,Favorite Training and reward treat,These are the BEST treats for training and rew...,1,these are the best treats for training and rew...


In [61]:
# perform text preprocessing
text_preprocess(df, ['html', 'punc', 'lower'])

In [62]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[-1, 0, 1], ordered=True)
df['Sentiment'] = df['Sentiment'].astype(cat_type)

In [65]:
df

,Id,Score,Summary,Text,Sentiment,Clean_text
0,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,i have bought several of the vitality canned d...
1,2,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,-1,product arrived labeled as jumbo salted peanut...
2,3,4,"""Delight"" says it all",This is a confection that has been around a fe...,0,this is a confection that has been around a fe...
3,4,2,Cough Medicine,If you are looking for the secret ingredient i...,-1,if you are looking for the secret ingredient i...
4,5,5,Great taffy,Great taffy at a great price. There was a wid...,1,great taffy at a great price there was a wide...
...,...,...,...,...,...,...
568422,568450,5,Will not do without,Great for sesame chicken..this is a good if no...,1,great for sesame chickenthis is a good if not ...
568423,568451,2,disappointed,I'm disappointed with the flavor. The chocolat...,-1,im disappointed with the flavor the chocolate ...
568424,568452,5,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o...",1,these stars are small so you can give 1015 of ...
568425,568453,5,Favorite Training and reward treat,These are the BEST treats for training and rew...,1,these are the best treats for training and rew...


In [66]:
X = df['Clean_text']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
X_train_counts, X_train_tfidf = vectorize(X_train)

In [71]:
#X_train_tfidf.toarray()
tfdf = pd.DataFrame()
tfdf['test'] = list(X_train_tfidf)

In [74]:
tfdf

,test
0,"(0, 7801)\t0.10482848367219183\n (0, 14923)..."
1,"(0, 18773)\t0.04463218426774518\n (0, 20012..."
2,"(0, 17664)\t0.17407765595569785\n (0, 21699..."
3,"(0, 22023)\t0.18257418583505536\n (0, 23299..."
4,"(0, 15160)\t0.16012815380508713\n (0, 23299..."
...,...
454736,"(0, 4257)\t0.019451424796105867\n (0, 14861..."
454737,"(0, 20012)\t0.11952286093343936\n (0, 21758..."
454738,"(0, 17811)\t0.08006407690254357\n (0, 20012..."
454739,"(0, 5203)\t0.06537204504606134\n (0, 14923)..."


In [53]:
from sklearn.pipeline import Pipeline
clf_ordReg_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_ordReg", OrderedModel(distr='logit'))])
clf_ordReg_pipe.fit(X_train, y_train)

TypeError: __init__() missing 2 required positional arguments: 'endog' and 'exog'

In [57]:
X_train_counts, X_train_tfidf = vectorize(X_train)

X_train_tfidf.toarray()

<1x1 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [75]:
mod_prob = OrderedModel(df['Sentiment'], tfdf['test'], distr='probit')
res_prob = mod_prob.fit()
res_prob.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [54]:
X_train_tfidf

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [31]:
from sklearn.model_selection import StratifiedShuffleSplit

data = df

print("Before {}".format(len(data)))
dataAfter = data.dropna(subset=["Score"]) # removes all NAN in reviews.rating
print("After {}".format(len(dataAfter)))
dataAfter["Score"] = dataAfter["Score"].astype(int)

split = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for train_index, test_index in split.split(dataAfter, dataAfter["Score"]): 
    strat_train = dataAfter.reindex(train_index)
    strat_test = dataAfter.reindex(test_index)

array([['tasty by itself but not much more sugar than regular peanut butter this stuff is delicious especially for those who love the salty sweetness of chocolate n peanut butter ive used this as if it were a sort of fudge for ice cream heat up one serving for about 30 seconds in the microwave and then throw the ice cream in and enjoy very yummy'],
       ['john wm macys crunchy snack sticks are great  i have loved john wm macys sweet sticks for years the chocolate sticks are my favorite along with the java cinnamon sticks if you like a light slightly sweet snack or some crunchy nibbles with your coffee you cant go wrong  my suggestion is to order in bulk the chocolate ones are not offered on amazon yet and the cheddar stick and snacks  are good as well  but i love the lightly sweetened cinnamon and chocolate ones better  they are great for snacking or with coffee and breakfast  most of all  i love the products because of the ingredients  im intollerant to margarines and palm oil produ

In [25]:
df[['Clean_text']]

,Clean_text
0,i have bought several of the vitality canned d...
1,product arrived labeled as jumbo salted peanut...
2,this is a confection that has been around a fe...
3,if you are looking for the secret ingredient i...
4,great taffy at a great price there was a wide...
...,...
568449,great for sesame chickenthis is a good if not ...
568450,im disappointed with the flavor the chocolate ...
568451,these stars are small so you can give 1015 of ...
568452,these are the best treats for training and rew...


In [26]:
[X_train]

[321645    tasty by itself but not much more sugar than r...
 72131     john wm macys crunchy snack sticks are great  ...
 80123     i have used this product for a long time as li...
 49098     i cant find this flavor in my local grocery an...
 317894    no apple flavor what so ever has a horrible me...
                                 ...                        
 110272    yes this is spendy for genmai cha the thing is...
 259188    these are great the texture is like no other f...
 365856    we give these several times a week to all our ...
 131937    but thankfully amazon is there to be my suppli...
 121963    we have four dogs and they have done very well...
 Name: Clean_text, Length: 454741, dtype: object]